In [1]:
# Hack to always autoreload modules and avoid restarting the kernel each time

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
# Note: This is a hack to allow importing from the parent directory
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))

In [3]:
import optuna
import torch
from models.gaussian_image_trainer import GaussianImageTrainer
from configs import Config
from utils.search_space import is_valid_combination

/home/h/harjesruiloba/miniforge3/envs/nerfstudio/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torchvision
from constants import CIFAR10_TRANSFORM
from utils.data import create_default_image
from utils import visualize_tensor

# Only used one image for the first tuning
dataset = torchvision.datasets.CIFAR10(root="./data", download=True)
# cifar_image = CIFAR10_TRANSFORM(dataset[2][0])
cifar_imgs = [CIFAR10_TRANSFORM(dataset[i][0]) for i in range(4)]
visualize_tensor(cifar_imgs[1])

Files already downloaded and verified


In [5]:
import logging
import sys

In [6]:
# New experiments with some key learnings:]
# 1. LR 0.01 seems to be the best.
# 2. Max Steps 2000 is better than 1000, larger values are not much better.
# 3. Init Type: KNN is the best, Grid is still interesting for other reasons (AE), don't use random. 
# 4. Model Type: 2dgs is superior
# 5. More points is better, but 32x32 is already very good.
# 6. Init Scale 2.0 is clearly the best, bigger values might also be good but were not tested.
# 7. Init Opacity between 0.5 and 0.75 is the best.
# 8. Scale Regulation 0.1 is the best.
# 9. Opacity Regulation 0.5 is the best.
# 10. Extent 2 and 4 are similar, 1 is worse.

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name_grid = "study-w-grid-2"  # Unique identifier of the study.
storage_name_grid = "sqlite:///{}.db".format(study_name_grid)
study_w_grid = optuna.create_study(study_name=study_name_grid, storage=storage_name_grid, load_if_exists=True)

Error: no "view" rule for type "image/png" passed its test case
       (for more information, add "--debug=1" on the command line)


[I 2025-01-08 22:47:28,837] Using an existing study with name 'study-w-grid-2' instead of creating a new one.


Using an existing study with name 'study-w-grid-2' instead of creating a new one.


In [11]:
def objective_w_grid_2(trial: optuna.Trial) -> float:
    """
    Objective function for Optuna hyperparameter optimization.

    Args:
        trial (optuna.Trial): The trial object used to suggest hyperparameters.

    Returns:
        float: The evaluation metric to minimize (e.g., validation loss).
    """
    # Conditional sampling
    # group_optimization = trial.suggest_categorical("group_optimization", [True, False])
    # strategy = None
    group_optimization = True # For now, as having default strategy was causing an error
    if not group_optimization:
        strategy = trial.suggest_categorical("strategy", ["default", "mcmc"])

    # Hyperparameter suggestions
    loss_weights = [
        1/3,
        1/3,
        1/3,
    ]

    scale_regularization = trial.suggest_categorical("scale_regularization", [None, 0.1, 0.5, 1.0])
    opacity_regularization = trial.suggest_categorical("opacity_regularization", [None, 0.1, 0.5, 1.0])

    # init_type = trial.suggest_categorical("init_type", ["grid", "knn"])
    init_type = "grid"  # Focus on the option with a fixed grid for this study
    num_points = 32*32
    extent = trial.suggest_categorical("extent", [1.0, 2.0, 4.0])
    init_scale = trial.suggest_categorical("init_scale", [2.0, 3.0, 4.0])
    init_opacity = trial.suggest_categorical("init_opacity", [0.5, 0.6, 0.7])

    max_steps = 1000
    learning_rate = 0.01

    model_type = "2dgs"  # Left out "2dgs-inria" as it needed further dependencies
    bilateral_grid = False

    image_index = trial.suggest_categorical("image_index", [0, 1, 2, 3])

    if not is_valid_combination({
        "learning_rate": learning_rate,
        "loss_weights": loss_weights,
        "group_optimization": group_optimization,
        # "strategy": strategy,
        "learning_rate": learning_rate,
        "model_type": model_type,
        "bilateral_grid": bilateral_grid,
    }, GaussianImageTrainer):
        raise optuna.exceptions.TrialPruned("Invalid hyperparameter combination")

    # Create Config object
    cfg = Config(
        
        seed=42,
        image=cifar_imgs[image_index],  # Replace with actual ground truth image tensor
        max_steps=max_steps,
        learning_rate=learning_rate,
        loss_weights=loss_weights,
        init_type=init_type,
        num_points=num_points,
        init_scale=init_scale,
        init_opacity=init_opacity,
        # Fixed, added these:
        scale_regularization=scale_regularization,
        opacity_regularization=opacity_regularization,
        extent=extent,
        group_optimization=group_optimization,
        # strategy=strategy,
        model_type=model_type,
        # bilateral_grid=bilateral_grid,
        bilateral_grid=False
    )

    # Initialize and train the model
    trainer = GaussianImageTrainer(cfg)
    result = trainer.train()

    # Evaluate generated image quality (e.g., L1 loss with ground truth)
    generated_image = result.cpu()  # Replace with actual rendered output
    ground_truth_image = cfg.image.cpu()
    # evaluation_metric = torch.nn.functional.l1_loss(generated_image, ground_truth_image)  # Old version
    evaluation_metric = trainer.l1_loss * 1/3 + trainer.mse_loss * 1/3 + trainer.ssim_loss * 1/3  # Simply take average as loss

    return evaluation_metric.item()

In [12]:
study_w_grid.optimize(objective_w_grid_2, n_trials=1000, n_jobs=1, timeout=3600)

[W 2025-01-08 22:55:22,348] Trial 2 failed with parameters: {'scale_regularization': None, 'opacity_regularization': 1.0, 'extent': 4.0, 'init_scale': 2.0, 'init_opacity': 0.5, 'image_index': 3} because of the following error: Exception('No GPU available. `gpsplat` requires a GPU to train.').
Traceback (most recent call last):
  File "/home/h/harjesruiloba/miniforge3/envs/nerfstudio/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/user/22473/ipykernel_2122290/3286051834.py", line 78, in objective_w_grid_2
    trainer = GaussianImageTrainer(cfg)
  File "/home/h/harjesruiloba/Projects/visual-representation-learning/models/gaussian_image_trainer.py", line 42, in __init__
    raise Exception("No GPU available. `gpsplat` requires a GPU to train.")
Exception: No GPU available. `gpsplat` requires a GPU to train.


Trial 2 failed with parameters: {'scale_regularization': None, 'opacity_regularization': 1.0, 'extent': 4.0, 'init_scale': 2.0, 'init_opacity': 0.5, 'image_index': 3} because of the following error: Exception('No GPU available. `gpsplat` requires a GPU to train.').
Traceback (most recent call last):
  File "/home/h/harjesruiloba/miniforge3/envs/nerfstudio/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/user/22473/ipykernel_2122290/3286051834.py", line 78, in objective_w_grid_2
    trainer = GaussianImageTrainer(cfg)
  File "/home/h/harjesruiloba/Projects/visual-representation-learning/models/gaussian_image_trainer.py", line 42, in __init__
    raise Exception("No GPU available. `gpsplat` requires a GPU to train.")
Exception: No GPU available. `gpsplat` requires a GPU to train.


[W 2025-01-08 22:55:22,351] Trial 2 failed with value None.


Trial 2 failed with value None.


Exception: No GPU available. `gpsplat` requires a GPU to train.